<a href="https://colab.research.google.com/github/tkhan3/airo/blob/master/hummingbird_cpu2gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install hummingbird-ml

     |████████████████████████████████| 81kB 2.1MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 7.4MB 26.2MB/s 


Create a fake dataset for a classification problem. Our dataset 50 variables with 100000 records in 3 classes.

Randomforest Classifier from Scikit-learn will be trained using this data.

In [15]:
import numpy as np
np.random.seed(42 )

In [17]:
from sklearn.ensemble import RandomForestClassifier
num_classes = 3
X = np.random.rand(100000, 50)
y = np.random.randint(num_classes, size=100000)
randomForestCPU = RandomForestClassifier(n_estimators=10, max_depth=10)
randomForestCPU.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
%%timeit
randomForestCPU.predict_proba(X)

10 loops, best of 3: 99.1 ms per loop


In [34]:
y_cpu = randomForestCPU.predict(X)

In [19]:
from hummingbird.ml import convert
# Use Hummingbird to convert the model to PyTorch
randomForestGPU = convert(randomForestCPU, 'pytorch')
randomForestGPU.to('cuda')

In [23]:
%%timeit
randomForestGPU.predict_proba(X)

10 loops, best of 3: 20.5 ms per loop


In [41]:
y_gpu = randomForestGPU.predict(X)
y_cpu = randomForestCPU.predict(X)

In [42]:
if False in np.equal(y_gpu,y_cpu).tolist():
  print ("prediction mismatched")
else:
  print ("prediction matched")

prediction matched


In [8]:
from sklearn.linear_model import LogisticRegression
num_classes = 3
X = np.random.rand(100000, 50)
y = np.random.randint(num_classes, size=100000)
LogisticRegressionCPU = LogisticRegression(random_state=42)
LogisticRegressionCPU.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
%%timeit
LogisticRegressionCPU.predict_proba(X)

10 loops, best of 3: 36.8 ms per loop


In [10]:
from hummingbird.ml import convert
# Use Hummingbird to convert the model to PyTorch
LogisticRegressionGPU = convert(LogisticRegressionCPU, 'pytorch')
LogisticRegressionGPU.to('cuda')

In [12]:
%%timeit
LogisticRegressionGPU.predict_proba(X)

100 loops, best of 3: 11.4 ms per loop
